In [2]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [3]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-TEI_XML_NLP'

### PART I: Examining Information on places

#### Reading the XML Metadata files

In [4]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [5]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [6]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [7]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [8]:
identified_places = collections.defaultdict(list)
def show_record(something):
    if isinstance(something, list):
        for elem in something:
            show_record(elem)
    elif isinstance(something, collections.OrderedDict) and ('record' in something.keys()):
        try:
            #print(something['record']['Bundesland'], ' --> ', something['record']['Ort'])
            identified_places[something['record']['Bundesland']].append(something['record']['Ort'])
        except:
            #print(something['record'][0]['Bundesland'], ' --> ', something['record'][0]['Ort'])
            identified_places[something['record'][0]['Bundesland']].append(something['record'][0]['Ort'])
    elif isinstance(something, collections.OrderedDict):
        new = something['listPlace']['place']
        show_record(new)
    else:
        print(type(something))
        print(something)

In [9]:
show_record(placeslist)

In [10]:
identified_places.keys()

dict_keys(['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.'])

In [11]:
df_places = pd.DataFrame(columns=['Bundesland', 'Ort'])
for key in identified_places.keys():
    df_temp = pd.DataFrame(identified_places[key], columns=['Ort'])
    df_temp['Bundesland'] = key
    df_places = pd.concat([df_places, df_temp], axis=0, sort=True)
df_places.head(30)

,Bundesland,Ort
0,W.,Altmannsdorf
1,W.,Aspern
2,W.,Atzgersdorf
3,W.,Breitensee
4,W.,Favoriten
5,W.,Gersthof
6,W.,Hernals
7,W.,Hietzing
8,W.,Inzersdorf
9,W.,Kalksburg


In [12]:
df_places.reset_index(inplace=True, drop=True)

In [13]:
df_places.replace('W.','Wien', inplace=True)
df_places.replace('STir.','Südtirol', inplace=True)
df_places.replace('OTir.','Osttiroler', inplace=True)
df_places.replace('NTir.','Nord tirol', inplace=True)
df_places.replace('Kä.','Kärnten', inplace=True)
df_places.replace('St.','Steiermark', inplace=True)
df_places.replace('Sa.','Salzburg', inplace=True)
df_places.replace('OÖ','Oberösterreich', inplace=True)
df_places.replace('NÖ','Niederösterreich', inplace=True)
df_places.replace('Bgl.','Burgenland', inplace=True)

In [14]:
df_places.head(10)

,Bundesland,Ort
0,Wien,Altmannsdorf
1,Wien,Aspern
2,Wien,Atzgersdorf
3,Wien,Breitensee
4,Wien,Favoriten
5,Wien,Gersthof
6,Wien,Hernals
7,Wien,Hietzing
8,Wien,Inzersdorf
9,Wien,Kalksburg


In [15]:
df_places.loc[0].values

array(['Wien', 'Altmannsdorf'], dtype=object)

#### Using a geolocator to get the Coordinates

In [17]:
geolocator = gg.Nominatim(user_agent="test_application")

In [18]:
df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']

'Altmannsdorf, Wien'

In [19]:
logradouro = df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Altmannsdorf, Hoffingergasse, KG Altmannsdorf, Meidling, Kabelwerk, Wien, 1120, Österreich
(48.16443205, 16.3217472203092)
{'place_id': 157414934, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 366749912, 'boundingbox': ['48.1554092', '48.1735359', '16.304118', '16.3372646'], 'lat': '48.16443205', 'lon': '16.3217472203092', 'display_name': 'Altmannsdorf, Hoffingergasse, KG Altmannsdorf, Meidling, Kabelwerk, Wien, 1120, Österreich', 'class': 'boundary', 'type': 'statistical', 'importance': 0.48123930027933803}


The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [20]:
def get_coordinates(place):
    try:
        location = geolocator.geocode(place)
    except:
        return None
    print(location)
    time.sleep(5)
    if location:
        return (location.longitude, location.latitude), location.address
    else:
        return None

In [21]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,Wien,Altmannsdorf,"(16.3217472203092, 48.16443205)","Altmannsdorf, Hoffingergasse, KG Altmannsdorf,..."
1,Wien,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,Wien,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,Wien,Breitensee,"(16.3064385, 48.2022735)","Breitensee, Penzing, Wien, 1140, Österreich"
4,Wien,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,Wien,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,Wien,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,Wien,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,Wien,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,Wien,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [22]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 4 columns):
Bundesland     2981 non-null object
Ort            2981 non-null object
Coordinates    2953 non-null object
Adresses       2953 non-null object
dtypes: object(4)
memory usage: 93.2+ KB


In [23]:
df_places.Bundesland.unique()

array(['Wien', 'Südtirol', 'Osttiroler', 'Nord tirol', 'Kärnten',
       'Steiermark', 'Salzburg', 'Oberösterreich', 'Niederösterreich',
       'Burgenland'], dtype=object)

In [24]:
df_places[df_places.Coordinates.isnull()]

,Bundesland,Ort,Coordinates,Adresses
215,Osttiroler,Lavant,None,None
290,Nord tirol,Feichten im Kaunertal (Feuchten),None,None
344,Nord tirol,Freundsheim,None,None
345,Nord tirol,Untermieming $Untermiening$,None,None
353,Nord tirol,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,Nord tirol,Thiersee $Tiersee$,None,None
484,Nord tirol,Aschau bei Kirchberg,None,None
552,Kärnten,Nölbing,None,None
571,Kärnten,Reisach im Gailtail,None,None
580,Kärnten,Kreuschlach,None,None


Testing with a missing field...

In [25]:
location = geolocator.geocode('Niederwallsee, Niederösterreich')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

AttributeError: 'NoneType' object has no attribute 'address'

In [26]:
df_places[df_places.Coordinates.isnull()].index

Int64Index([ 215,  290,  344,  345,  353,  446,  484,  552,  571,  580,  761,
             830,  854, 1037, 1199, 1241, 1281, 1293, 1332, 1615, 1773, 1865,
            2008, 2012, 2265, 2305, 2566, 2971],
           dtype='int64')

In [27]:
geoinfo_missing = []
for ort in df_places[df_places.Coordinates.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [28]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Coordinates.isnull()]))

28
28


In [29]:
idx = df_places[df_places.Coordinates.isnull()].index
df_places.loc[idx, 'Coordinates'] = [x[0] if x else None for x in geoinfo_missing]

In [30]:
df_places[df_places.Adresses.isnull()]

,Bundesland,Ort,Coordinates,Adresses
215,Osttiroler,Lavant,None,None
290,Nord tirol,Feichten im Kaunertal (Feuchten),None,None
344,Nord tirol,Freundsheim,None,None
345,Nord tirol,Untermieming $Untermiening$,None,None
353,Nord tirol,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,Nord tirol,Thiersee $Tiersee$,None,None
484,Nord tirol,Aschau bei Kirchberg,None,None
552,Kärnten,Nölbing,None,None
571,Kärnten,Reisach im Gailtail,None,None
580,Kärnten,Kreuschlach,None,None


In [31]:
geoinfo_missing = []
for ort in df_places[df_places.Adresses.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [32]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Adresses.isnull()]))

28
28


In [33]:
idx = df_places[df_places.Adresses.isnull()].index
df_places.loc[idx, 'Adresses'] = [x[1] if x else None for x in geoinfo_missing]

In [34]:
df_places.loc[df_places[df_places.Coordinates.isnull()].index]

,Bundesland,Ort,Coordinates,Adresses
215,Osttiroler,Lavant,None,None
290,Nord tirol,Feichten im Kaunertal (Feuchten),None,None
344,Nord tirol,Freundsheim,None,None
345,Nord tirol,Untermieming $Untermiening$,None,None
353,Nord tirol,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,Nord tirol,Thiersee $Tiersee$,None,None
484,Nord tirol,Aschau bei Kirchberg,None,None
552,Kärnten,Nölbing,None,None
571,Kärnten,Reisach im Gailtail,None,None
580,Kärnten,Kreuschlach,None,None


#### Showing places in a map

In [16]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,Wien,Altmannsdorf,"(16.3217472203092, 48.16443205)","Altmannsdorf, Hoffingergasse, KG Altmannsdorf,..."
1,Wien,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,Wien,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,Wien,Breitensee,"(16.3064385, 48.2022735)","Breitensee, Penzing, Wien, 1140, Österreich"
4,Wien,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,Wien,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,Wien,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,Wien,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,Wien,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,Wien,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [17]:
df_places.Coordinates[1][-1::-1]

(48.2181854, 16.4847508)

In [18]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64
#http://andrewgaidus.com/leaflet_webmaps_python/

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx in df_places.index:
    if df_places.loc[idx,'Coordinates']:
        title = df_places.loc[idx,'Ort']
        markers.append(ipyleaflet.Marker(location=df_places.loc[idx,'Coordinates'][-1::-1], draggable=False, title=title))                    
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [19]:
#using Folio

mapplaces = folium.Map(location=(47,14), zoom_start=4)

mc = folium.plugins.MarkerCluster()

for idx in df_places.index:
    if df_places.loc[idx,'Coordinates']:
        title = df_places.loc[idx,'Ort']
        mc.add_child(folium.Marker(location=df_places.loc[idx,'Coordinates'][-1::-1], popup=title))
                       
mapplaces.add_child(mc)
mapplaces